In [1]:
import pandas as pd
import numpy as np

In [2]:
#Using matches_final.csv and merged_lineups_2022.csv
mfn = pd.read_csv('matches_final.csv')
lineup = pd.read_csv('merged_lineups_2022.csv')
ppt = 11 #Player per team
statsCount = 36 #Number of stats used
lineup

,team_id,fixture_id,player_id,player_name,number,position,date,club,Height,Weight,...,Curve,FKAcc.,Penalties,Volleys,GKPositioning,GKDiving,GKHandling,GKKicking,GKReflexes,Overall
0,19,18138812,9438,Aaron Ramsdale,32,G,1/1/2022,Arsenal,191,77,...,17,16,25,16,75,80,75,80,80,78
1,19,18138812,172960,Kieran Tierney,3,D,1/1/2022,Arsenal,178,70,...,72,56,42,38,12,13,16,8,13,80
2,19,18138812,310288,Takehiro Tomiyasu,18,D,1/1/2022,Arsenal,188,84,...,48,37,37,30,5,9,14,6,11,77
3,19,18138812,97811,Gabriel Magalhaes,6,D,1/1/2022,Arsenal,190,87,...,47,27,40,28,10,12,8,13,7,80
4,19,18138812,3259,Ben White,4,D,1/1/2022,Arsenal,185,77,...,26,20,24,21,5,8,11,12,6,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13481,485,18165743,61186,William Carvalho,14,M,15/04/2022,Real Betis,187,79,...,80,50,60,50,14,6,6,5,14,79
13482,485,18165743,254081,Guido Rodriguez,21,M,15/04/2022,Real Betis,185,80,...,49,50,53,47,16,12,11,14,6,78
13483,485,18165743,1410,Juanmi,7,M,15/04/2022,Real Betis,172,69,...,62,57,64,71,12,11,6,9,9,75
13484,485,18165743,96125,Nabil Fekir,8,M,15/04/2022,Real Betis,173,75,...,83,79,74,77,14,15,9,8,15,85


In [3]:
numsLnup = ['team_id', 'fixture_id', 'Height', 'Weight', 'BallControl', 'Marking', 'Dribbling', 'SlideTackle',
 'StandTackle', 'Aggression', 'Reactions', 'Att.Position', 'Interceptions', 'Vision', 'Crossing', 'ShortPass', 'LongPass',
 'Acceleration', 'Stamina', 'Strength', 'Balance', 'SprintSpeed', 'Agility', 'Jumping', 'Heading', 'ShotPower', 'Finishing',
 'LongShots', 'Curve', 'FKAcc.', 'Penalties', 'Volleys', 'GKPositioning', 'GKDiving', 'GKHandling', 'GKKicking',
 'GKReflexes', 'Overall']
numsCols = lineup[numsLnup]
colsName = ['id','home_id','away_id']
for i in range(statsCount):
    colsName.append('gap_stat_' + str(i+1))

iptStats = ['Vision','LongPass','SprintSpeed','Acceleration','Finishing' ,'LongShots', 'Curve']
for i in iptStats:
    colsName.append('gap_max_'+ i)

FormStats = pd.DataFrame(columns=colsName)

for i in range(int(lineup.shape[0]/(2*ppt))):
    sum = 0
    for j in range(2*ppt):
        if numsCols['team_id'][i*11+ j] == mfn[mfn['id'] == numsCols.fixture_id.iloc[i*11+ j]]['localteam_id'].values[0]:
            sum = np.array(sum + numsCols.iloc[i*11+ j, 2:].values)
        else: 
            sum = np.array(sum - numsCols.iloc[i*11+ j, 2:].values)
    sum = sum/11

    for j in iptStats:
        gapMaxStat = 0
        for k in range(2):
            if numsCols['team_id'][(i+k)*11] == mfn[mfn['id'] == numsCols.fixture_id.iloc[i*11]]['localteam_id'].values[0]:
                gapMaxStat = gapMaxStat + numsCols[j].iloc[(i+k)*11: (i+k+1)*11].max()
            else:
                gapMaxStat = gapMaxStat - numsCols[j].iloc[(i+k)*11: (i+k+1)*11].max()
        sum = np.append(sum, gapMaxStat)
    match_id = int(numsCols.fixture_id.iloc[i*22])
    home_id = int(mfn[mfn['id'] == numsCols.fixture_id.iloc[i*22]]['localteam_id'].values[0])
    away_id = int(mfn[mfn['id'] == numsCols.fixture_id.iloc[i*22]]['visitorteam_id'].values[0])
        
    ids = np.array([match_id, home_id, away_id])
    sum = np.concatenate((ids, sum), axis= None)
    FormStats.loc[i] = sum
FormStats[['id','home_id','away_id']] = FormStats[['id','home_id','away_id']].astype(int)
FormStats


,id,home_id,away_id,gap_stat_1,gap_stat_2,gap_stat_3,gap_stat_4,gap_stat_5,gap_stat_6,gap_stat_7,...,gap_stat_34,gap_stat_35,gap_stat_36,gap_max_Vision,gap_max_LongPass,gap_max_SprintSpeed,gap_max_Acceleration,gap_max_Finishing,gap_max_LongShots,gap_max_Curve
0,18138812,19,9,1.363636,0.727273,-6.909091,-0.545455,-5.636364,4.818182,0.181818,...,-0.636364,-1.545455,-6.545455,-8.0,-8.0,-2.0,-7.0,0.0,-13.0,-2.0
1,18138821,25,6,4.181818,5.090909,-13.454545,-4.909091,-13.909091,0.909091,-0.363636,...,-1.818182,-2.818182,-12.000000,-170.0,-169.0,-173.0,-184.0,-159.0,-170.0,-160.0
2,18138815,51,1,2.545455,1.454545,-7.363636,-4.272727,-8.909091,-5.272727,0.000000,...,0.545455,-4.000000,-7.909091,-11.0,-10.0,-5.0,-3.0,-18.0,-11.0,-10.0
3,18138813,236,15,2.545455,0.363636,-4.818182,-4.181818,-6.181818,-6.909091,-1.636364,...,2.000000,-0.909091,-6.363636,-167.0,-166.0,-174.0,-176.0,-173.0,-167.0,-160.0
4,18138816,13,78,4.272727,2.727273,-0.454545,0.727273,-1.454545,-0.090909,1.272727,...,1.818182,1.272727,-2.545455,2.0,2.0,-3.0,5.0,-2.0,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,18165752,528,36,-0.363636,1.090909,-12.363636,-5.090909,-9.090909,-3.090909,-2.181818,...,0.454545,-0.090909,-6.636364,-14.0,-9.0,9.0,7.0,1.0,-12.0,-10.0
609,18165759,1099,594,-3.090909,-1.909091,-1.272727,-2.727273,4.636364,-4.545455,-4.272727,...,1.909091,2.181818,-0.181818,-171.0,-159.0,-170.0,-167.0,-156.0,-157.0,-164.0
610,18165753,3457,83,-3.545455,-3.909091,3.909091,1.181818,4.818182,-0.181818,-1.636364,...,1.727273,3.909091,1.181818,6.0,-1.0,-2.0,-6.0,-3.0,-1.0,6.0
611,18165757,377,214,-0.454545,1.545455,-8.000000,-6.818182,-7.272727,-3.272727,-4.636364,...,0.727273,1.272727,-5.818182,-150.0,-152.0,-168.0,-169.0,-158.0,-156.0,-156.0


In [4]:
FormStats.to_csv('FormStatsDone.csv')